In [1]:
print("PREPARANDO EL ENTORNO\n\n")
import os
 # Instalar SDK java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Descargar Spark
!wget -q https://archive.apache.org/dist/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
# Descomprimir la versión de Spark
!tar xf spark-3.3.1-bin-hadoop3.tgz
# Establecer las variables de entorno
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"
# Descargar findspark
!pip install -q findspark
# Descargar pyspark
!pip install -q pyspark
print("\n\n******** INSTALACIÓN CORRECTA *******")

PREPARANDO EL ENTORNO


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 19.0 MB/s eta 0:00:00


******** INSTALACIÓN CORRECTA *******


In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [ ]:
data = spark.read.parquet("./data80/convertir/")

In [ ]:
data.printSchema()

root
 |-- date: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- date_str: string (nullable = true)
 |-- ts_str: string (nullable = true)



In [ ]:
data.show(truncate=False)

+----------+-----------------------+----------+----------------+
|date      |timestamp              |date_str  |ts_str          |
+----------+-----------------------+----------+----------------+
|2021-01-01|2021-01-01 20:10:50.723|01-01-2021|18-08-2021 46:58|
+----------+-----------------------+----------+----------------+



**Funciones de fecha y hora**

In [ ]:
from pyspark.sql.functions import col, to_date, to_timestamp
data1 = data.select(
    to_date(col('date')).alias('date1'),
    to_timestamp(col('timestamp')).alias('ts1'),
    to_date(col('date_str'), 'dd-MM-yyyy').alias('date2'),
    to_timestamp(col('ts_str'), 'dd-MM-yyyy mm:ss').alias('ts2')
)

In [ ]:
data1.show(truncate=False)

+----------+-----------------------+----------+-------------------+
|date1     |ts1                    |date2     |ts2                |
+----------+-----------------------+----------+-------------------+
|2021-01-01|2021-01-01 20:10:50.723|2021-01-01|2021-08-18 00:46:58|
+----------+-----------------------+----------+-------------------+



In [ ]:
data1.printSchema()

root
 |-- date1: date (nullable = true)
 |-- ts1: timestamp (nullable = true)
 |-- date2: date (nullable = true)
 |-- ts2: timestamp (nullable = true)



In [ ]:
from pyspark.sql.functions import date_format

In [ ]:
# Le damos otro formato
data1.select(
    date_format(col('date1'), 'dd-MM-yyyy')
).show()

+------------------------------+
|date_format(date1, dd-MM-yyyy)|
+------------------------------+
|                    01-01-2021|
+------------------------------+



In [ ]:
df = spark.read.parquet("./data80/calculo/")

In [ ]:
df.show()

+------+-------------+------------+-------------------+
|nombre|fecha_ingreso|fecha_salida|       baja_sistema|
+------+-------------+------------+-------------------+
|  Jose|   2021-01-01|  2021-11-14|2021-10-14 15:35:59|
|Mayara|   2021-02-06|  2021-11-25|2021-11-25 10:35:55|
+------+-------------+------------+-------------------+



In [ ]:
from pyspark.sql.functions import datediff, months_between, last_day

In [ ]:
# Operaciones con fecha
df.select(
    col('nombre'), 
    datediff(col('fecha_salida'), col('fecha_ingreso')).alias('dias'), 
    months_between(col('fecha_salida'), col('fecha_ingreso')).alias('meses'), 
    last_day(col('fecha_salida')).alias('ultimo_dia_mes')
).show()

+------+----+-----------+--------------+
|nombre|dias|      meses|ultimo_dia_mes|
+------+----+-----------+--------------+
|  Jose| 317|10.41935484|    2021-11-30|
|Mayara| 292| 9.61290323|    2021-11-30|
+------+----+-----------+--------------+



In [ ]:
from pyspark.sql.functions import date_add, date_sub

In [ ]:
# Operaciones con fecha
df.select(
    col('nombre'), 
    col('fecha_ingreso'), 
    date_add(col('fecha_ingreso'), 14).alias('mas_14_dias'), 
    date_sub(col('fecha_ingreso'), 1).alias('menos_1_dia')
).show()

+------+-------------+-----------+-----------+
|nombre|fecha_ingreso|mas_14_dias|menos_1_dia|
+------+-------------+-----------+-----------+
|  Jose|   2021-01-01| 2021-01-15| 2020-12-31|
|Mayara|   2021-02-06| 2021-02-20| 2021-02-05|
+------+-------------+-----------+-----------+



In [ ]:
from pyspark.sql.functions import year, month,dayofmonth, dayofyear, hour, minute, second

In [ ]:
df.select(
    col('baja_sistema'), 
    year(col('baja_sistema')),
    month(col('baja_sistema')), 
    dayofmonth(col('baja_sistema')), 
    dayofyear(col('baja_sistema')),
    hour(col('baja_sistema')), 
    minute(col('baja_sistema')), 
    second(col('baja_sistema'))
).show()

+-------------------+------------------+-------------------+------------------------+-----------------------+------------------+--------------------+--------------------+
|       baja_sistema|year(baja_sistema)|month(baja_sistema)|dayofmonth(baja_sistema)|dayofyear(baja_sistema)|hour(baja_sistema)|minute(baja_sistema)|second(baja_sistema)|
+-------------------+------------------+-------------------+------------------------+-----------------------+------------------+--------------------+--------------------+
|2021-10-14 15:35:59|              2021|                 10|                      14|                    287|                15|                  35|                  59|
|2021-11-25 10:35:55|              2021|                 11|                      25|                    329|                10|                  35|                  55|
+-------------------+------------------+-------------------+------------------------+-----------------------+------------------+-----------------

**Funciones para trabajar con strings**

In [ ]:
data = spark.read.parquet("./data81/")

In [ ]:
data.show()

+-------+
| nombre|
+-------+
| Spark |
+-------+



In [ ]:
from pyspark.sql.functions import ltrim, rtrim, trim

In [ ]:
# Eliminar espacios
data.select(
    ltrim('nombre').alias('ltrim'),
    rtrim('nombre').alias('rtrim'), 
    trim('nombre').alias('trim')
).show()

+------+------+-----+
| ltrim| rtrim| trim|
+------+------+-----+
|Spark | Spark|Spark|
+------+------+-----+



In [ ]:
from pyspark.sql.functions import col, lpad, rpad
data.select(
    trim(col('nombre')).alias('trim')
).select(
    lpad(col('trim'), 8, '-').alias('lpaqd'), 
    rpad(col('trim'), 8, '=').alias('rpad')
).show()

+--------+--------+
|   lpaqd|    rpad|
+--------+--------+
|---Spark|Spark===|
+--------+--------+



In [ ]:
df1 = spark.createDataFrame([('Spark', 'es', 'maravilloso')], ('sujeto', 'verbo', 'adjetivo'))

In [ ]:
df1.show()

+------+-----+-----------+
|sujeto|verbo|   adjetivo|
+------+-----+-----------+
| Spark|   es|maravilloso|
+------+-----+-----------+



In [ ]:
from pyspark.sql.functions import concat_ws, lower, upper, initcap, reverse

In [ ]:
# Transformación texto
df1.select(
    concat_ws(' ', col('sujeto'), col('verbo'), col('adjetivo')).alias('frase')
).select(
    col('frase'),
    lower(col('frase')).alias('minuscula'), 
    upper(col('frase')).alias('mayuscula'), 
    initcap(col('frase')).alias('initcap'), 
    reverse(col('frase')).alias('reversa')
).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|               frase|           minuscula|           mayuscula|             initcap|             reversa|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Spark es maravilloso|spark es maravilloso|SPARK ES MARAVILLOSO|Spark Es Maravilloso|osollivaram se krapS|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [ ]:
from pyspark.sql.functions import regexp_replace

In [ ]:
df2 = spark.createDataFrame([('voy a casa por mis llaves',)], ['frase'])

In [ ]:
df2.show(truncate=False)

+-------------------------+
|frase                    |
+-------------------------+
|voy a casa por mis llaves|
+-------------------------+



In [ ]:
# Reemplaza 'voy y por' por 'ir'
df2.select(
    regexp_replace(col('frase'), 'voy|por', 'ir').alias('nueva_frase')
).show(truncate=False)

+-----------------------+
|nueva_frase            |
+-----------------------+
|ir a casa ir mis llaves|
+-----------------------+



**Funciones para el trabajo con colecciones**

In [ ]:
data = spark.read.parquet("./data82/parquet/")

In [ ]:
data.show(truncate=False)

+-----+--------------------------------------------+
|dia  |tareas                                      |
+-----+--------------------------------------------+
|lunes|[hacer la tarea, buscar agua, lavar el auto]|
+-----+--------------------------------------------+



In [ ]:
data.printSchema()

root
 |-- dia: string (nullable = true)
 |-- tareas: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
from pyspark.sql.functions import col, size, sort_array, array_contains

In [ ]:
data.select(
    size(col('tareas')).alias('tamaño'), 
    sort_array(col('tareas')).alias('arreglo_ordenado'), 
    array_contains(col('tareas'), 'buscar agua').alias('buscar_agua')
).show(truncate=False)

# Tamaño arreglo | ordena el arreglo | busca texto

+------+--------------------------------------------+-----------+
|tamaño|arreglo_ordenado                            |buscar_agua|
+------+--------------------------------------------+-----------+
|3     |[buscar agua, hacer la tarea, lavar el auto]|true       |
+------+--------------------------------------------+-----------+



In [ ]:
from pyspark.sql.functions import explode

In [ ]:
# Separa el
data.select(
    col('dia'), 
    explode(col('tareas')).alias('tareas')
).show()

+-----+--------------+
|  dia|        tareas|
+-----+--------------+
|lunes|hacer la tarea|
|lunes|   buscar agua|
|lunes| lavar el auto|
+-----+--------------+



In [ ]:
# Formato JSON

In [3]:
json_df_str = spark.read.parquet("./data82/JSON/")

In [4]:
json_df_str.show(truncate=False)

+---------------------------------------------------------------------------+
|tareas_str                                                                 |
+---------------------------------------------------------------------------+
|{"dia": "lunes","tareas": ["hacer la tarea","buscar agua","lavar el auto"]}|
+---------------------------------------------------------------------------+



In [5]:
json_df_str.printSchema()

root
 |-- tareas_str: string (nullable = true)



In [9]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

schema_json = StructType(
    [
     StructField('dia', StringType(), True), 
     StructField('tareas', ArrayType(StringType()), True)
    ]
)

In [12]:
from pyspark.sql.functions import from_json, to_json, col

In [13]:
json_df = json_df_str.select(
    from_json(col('tareas_str'), schema_json).alias('por_hacer')
)

In [14]:
json_df.printSchema()

root
 |-- por_hacer: struct (nullable = true)
 |    |-- dia: string (nullable = true)
 |    |-- tareas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)



In [16]:
json_df.select(
    col('por_hacer').getItem('dia'),
    col('por_hacer').getItem('tareas'), 
    col('por_hacer').getItem('tareas').getItem(0).alias('primera_tarea')
).show(truncate=False)

+-------------+--------------------------------------------+--------------+
|por_hacer.dia|por_hacer.tareas                            |primera_tarea |
+-------------+--------------------------------------------+--------------+
|lunes        |[hacer la tarea, buscar agua, lavar el auto]|hacer la tarea|
+-------------+--------------------------------------------+--------------+



In [17]:
 json_df.select(
     to_json(col('por_hacer'))
 ).show(truncate=False)

+-------------------------------------------------------------------------+
|to_json(por_hacer)                                                       |
+-------------------------------------------------------------------------+
|{"dia":"lunes","tareas":["hacer la tarea","buscar agua","lavar el auto"]}|
+-------------------------------------------------------------------------+



**Funciones when, coalesce y lit**

In [19]:
data = spark.read.parquet("./data83/")

In [20]:
data.show()

+------+----+
|nombre|pago|
+------+----+
|  Jose|   1|
| Julia|   2|
| Katia|   1|
|  null|   3|
|  Raul|   3|
+------+----+



In [21]:
from pyspark.sql.functions import col, when, lit, coalesce

In [22]:
data.select(
    col('nombre'), 
    when(col('pago') == 1, 'pagado').when(col('pago') == 2, "sin pagar").otherwise("sin inicicar").alias('pago')
).show()

+------+------------+
|nombre|        pago|
+------+------------+
|  Jose|      pagado|
| Julia|   sin pagar|
| Katia|      pagado|
|  null|sin inicicar|
|  Raul|sin inicicar|
+------+------------+



In [24]:
data.select(
    coalesce(col('nombre'), lit('sin_nombre')).alias('nombre')
).show()

+----------+
|    nombre|
+----------+
|      Jose|
|     Julia|
|     Katia|
|sin_nombre|
|      Raul|
+----------+



**Funciones definidas por el usuario - UDF**


In [26]:
def f_cubo(n):
   return n * n * n

In [27]:
# Registramos la función
from pyspark.sql.types import LongType
spark.udf.register('cubo', f_cubo, LongType())

<function __main__.f_cubo(n)>

In [31]:
# Generamos DF usando la función f_cubo

# Generamos la vista
spark.range(1,10).createOrReplaceTempView('df_temp')
# Generamos la consulta
spark.sql("SELECT id, cubo(id) AS cubo FROM df_temp").show()

+---+----+
| id|cubo|
+---+----+
|  1|   1|
|  2|   8|
|  3|  27|
|  4|  64|
|  5| 125|
|  6| 216|
|  7| 343|
|  8| 512|
|  9| 729|
+---+----+



In [32]:
# Otea forma
def bienvenia(nombre):
  return ("hola {}".format(nombre))

In [33]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [34]:
bienvenida_udf = udf(lambda x: bienvenia(x), StringType())

In [35]:
df_nombre = spark.createDataFrame([('jose',), ('julia',)], ['nombre'])

In [36]:
df_nombre.show()

+------+
|nombre|
+------+
|  jose|
| julia|
+------+



In [37]:
from pyspark.sql.functions import col

In [38]:
df_nombre.select(
    col('nombre'), 
    bienvenida_udf(col('nombre')).alias('bie_nombre')
).show()

+------+----------+
|nombre|bie_nombre|
+------+----------+
|  jose| hola jose|
| julia|hola julia|
+------+----------+



In [39]:
# Otra forma
@udf(returnType=StringType())
def mayuscula(s):
  return s.upper()

In [40]:
df_nombre.select(
    col('nombre'), 
    mayuscula(col('nombre')).alias('may_nombre')
).show()

+------+----------+
|nombre|may_nombre|
+------+----------+
|  jose|      JOSE|
| julia|     JULIA|
+------+----------+



In [42]:
# Otra forma
import pandas as pd
from pyspark.sql.functions import pandas_udf

In [43]:
def cubo_pandas(a: pd.Series) -> pd.Series:
  return a * a * a

In [44]:
cubo_udf = pandas_udf(cubo_pandas, returnType=LongType())

In [45]:
x = pd.Series([1,2,3])

In [46]:
print(cubo_pandas(x))

0     1
1     8
2    27
dtype: int64


In [47]:
df = spark.range(5)

In [48]:
df.select(
    col('id'), 
    cubo_udf(col('id')).alias('cubo_pandas')
).show()

+---+-----------+
| id|cubo_pandas|
+---+-----------+
|  0|          0|
|  1|          1|
|  2|          8|
|  3|         27|
|  4|         64|
+---+-----------+



**Funciones de ventana**

In [49]:
df = spark.read.parquet("./data85/")

In [50]:
df.show()

+-------+----+------------+----------+
| nombre|edad|departamento|evaluacion|
+-------+----+------------+----------+
| Lazaro|  45|      letras|        98|
|   Raul|  24|  matemática|        76|
|  Maria|  34|  matemática|        27|
|   Jose|  30|     química|        78|
| Susana|  51|     química|        98|
|   Juan|  44|      letras|        89|
|  Julia|  55|      letras|        92|
|  Kadir|  38|arquitectura|        39|
| Lilian|  23|arquitectura|        94|
|   Rosa|  26|      letras|        91|
|   Aian|  50|  matemática|        73|
|Yaneisy|  29|      letras|        89|
|Enrique|  40|     química|        92|
|    Jon|  25|arquitectura|        78|
|  Luisa|  39|arquitectura|        94|
+-------+----+------------+----------+



In [51]:
# La función de ventanas operan en un grupo de filas y devuelve un valor único para cada fila

In [52]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, row_number, rank, dense_rank, col

In [58]:
windowSpec = Window.partitionBy('departamento').orderBy(desc('evaluacion'))

In [59]:
# row_number - Se usa para dar el número de fila secuencial, comenzando desde uno hasta
# el resultado de cada partición de ventana.

In [60]:
# Se crea una nueva columna "row_number", se le aplica la función de ventana, y se filtra solo 1 y 2.
df.withColumn('row_number', row_number().over(windowSpec)).filter(col('row_number').isin(1,2)).show()

+-------+----+------------+----------+----------+
| nombre|edad|departamento|evaluacion|row_number|
+-------+----+------------+----------+----------+
| Lilian|  23|arquitectura|        94|         1|
|  Luisa|  39|arquitectura|        94|         2|
| Lazaro|  45|      letras|        98|         1|
|  Julia|  55|      letras|        92|         2|
|   Raul|  24|  matemática|        76|         1|
|   Aian|  50|  matemática|        73|         2|
| Susana|  51|     química|        98|         1|
|Enrique|  40|     química|        92|         2|
+-------+----+------------+----------+----------+



In [62]:
# Rank - Se usa para proporcionar una rango al resultado dentro de una partición de ventana. Esta
# función deja huecos en el rango cuando hay empates.

In [63]:
df.withColumn('rank', rank().over(windowSpec)).show()

+-------+----+------------+----------+----+
| nombre|edad|departamento|evaluacion|rank|
+-------+----+------------+----------+----+
| Lilian|  23|arquitectura|        94|   1|
|  Luisa|  39|arquitectura|        94|   1|
|    Jon|  25|arquitectura|        78|   3|
|  Kadir|  38|arquitectura|        39|   4|
| Lazaro|  45|      letras|        98|   1|
|  Julia|  55|      letras|        92|   2|
|   Rosa|  26|      letras|        91|   3|
|   Juan|  44|      letras|        89|   4|
|Yaneisy|  29|      letras|        89|   4|
|   Raul|  24|  matemática|        76|   1|
|   Aian|  50|  matemática|        73|   2|
|  Maria|  34|  matemática|        27|   3|
| Susana|  51|     química|        98|   1|
|Enrique|  40|     química|        92|   2|
|   Jose|  30|     química|        78|   3|
+-------+----+------------+----------+----+



In [64]:
# dense_Rank - Simila a rank pero no deja huecos.

In [65]:
df.withColumn('dense_rank', dense_rank().over(windowSpec)).show()

+-------+----+------------+----------+----------+
| nombre|edad|departamento|evaluacion|dense_rank|
+-------+----+------------+----------+----------+
| Lilian|  23|arquitectura|        94|         1|
|  Luisa|  39|arquitectura|        94|         1|
|    Jon|  25|arquitectura|        78|         2|
|  Kadir|  38|arquitectura|        39|         3|
| Lazaro|  45|      letras|        98|         1|
|  Julia|  55|      letras|        92|         2|
|   Rosa|  26|      letras|        91|         3|
|   Juan|  44|      letras|        89|         4|
|Yaneisy|  29|      letras|        89|         4|
|   Raul|  24|  matemática|        76|         1|
|   Aian|  50|  matemática|        73|         2|
|  Maria|  34|  matemática|        27|         3|
| Susana|  51|     química|        98|         1|
|Enrique|  40|     química|        92|         2|
|   Jose|  30|     química|        78|         3|
+-------+----+------------+----------+----------+



In [66]:
# agregaciones con funciones de ventana

In [67]:
windowSpecAgg = Window.partitionBy('departamento')

In [68]:
from pyspark.sql.functions import min, max, avg

In [69]:
(
    df.withColumn('min', min(col('evaluacion')).over(windowSpecAgg))
    .withColumn('max', max(col('evaluacion')).over(windowSpecAgg))
    .withColumn('avg', avg(col('evaluacion')).over(windowSpecAgg))
    .withColumn('row_number', row_number().over(windowSpec))
).show()

+-------+----+------------+----------+---+---+------------------+----------+
| nombre|edad|departamento|evaluacion|min|max|               avg|row_number|
+-------+----+------------+----------+---+---+------------------+----------+
| Lilian|  23|arquitectura|        94| 39| 94|             76.25|         1|
|  Luisa|  39|arquitectura|        94| 39| 94|             76.25|         2|
|    Jon|  25|arquitectura|        78| 39| 94|             76.25|         3|
|  Kadir|  38|arquitectura|        39| 39| 94|             76.25|         4|
| Lazaro|  45|      letras|        98| 89| 98|              91.8|         1|
|  Julia|  55|      letras|        92| 89| 98|              91.8|         2|
|   Rosa|  26|      letras|        91| 89| 98|              91.8|         3|
|   Juan|  44|      letras|        89| 89| 98|              91.8|         4|
|Yaneisy|  29|      letras|        89| 89| 98|              91.8|         5|
|   Raul|  24|  matemática|        76| 27| 76|58.666666666666664|         1|

**Catalyst Optimizer**

"La forma más fácil de escribir aplicaciones de procesamiento de datos eficientes es no preocuparse por ello y optimizar automáticamente sus aplicaciones de procesamiento de datos."

In [71]:
data = spark.read.parquet("./data86/")

In [72]:
nuevo_df = (data.filter(col('MONTH').isin(6,7,8))
            .withColumn('dis_tiempo_aire', col('DISTANCE') / col('AIR_TIME'))
).select(
    col('AIRLINE'),
    col('dis_tiempo_aire')
).where(col('AIRLINE').isin('AA', 'DL', 'AS'))

In [73]:
nuevo_df.show()

+-------+-----------------+
|AIRLINE|  dis_tiempo_aire|
+-------+-----------------+
|     AA|7.649769585253456|
|     DL| 8.14176245210728|
|     DL|8.361344537815127|
|     DL|8.358851674641148|
|     AA|8.610294117647058|
|     AS|7.413461538461538|
|     AA|              8.5|
|     DL| 8.10135135135135|
|     DL|8.779005524861878|
|     DL| 8.08955223880597|
|     AS|8.274038461538462|
|     DL| 8.53048780487805|
|     DL| 8.11111111111111|
|     DL|8.434065934065934|
|     DL|8.388235294117647|
|     AS|8.089385474860336|
|     DL|8.273885350318471|
|     AA|7.700729927007299|
|     DL| 8.61504424778761|
|     AA|7.967741935483871|
+-------+-----------------+
only showing top 20 rows



In [74]:
# Mostramos el plan físico y plan lógico
nuevo_df.explain(True)

== Parsed Logical Plan ==
'Filter 'AIRLINE IN (AA,DL,AS)
+- Project [AIRLINE#439, dis_tiempo_aire#498]
   +- Project [YEAR#435, MONTH#436, DAY#437, DAY_OF_WEEK#438, AIRLINE#439, FLIGHT_NUMBER#440, TAIL_NUMBER#441, ORIGIN_AIRPORT#442, DESTINATION_AIRPORT#443, SCHEDULED_DEPARTURE#444, DEPARTURE_TIME#445, DEPARTURE_DELAY#446, TAXI_OUT#447, WHEELS_OFF#448, SCHEDULED_TIME#449, ELAPSED_TIME#450, AIR_TIME#451, DISTANCE#452, WHEELS_ON#453, TAXI_IN#454, SCHEDULED_ARRIVAL#455, ARRIVAL_TIME#456, ARRIVAL_DELAY#457, DIVERTED#458, ... 8 more fields]
      +- Filter MONTH#436 IN (6,7,8)
         +- Relation [YEAR#435,MONTH#436,DAY#437,DAY_OF_WEEK#438,AIRLINE#439,FLIGHT_NUMBER#440,TAIL_NUMBER#441,ORIGIN_AIRPORT#442,DESTINATION_AIRPORT#443,SCHEDULED_DEPARTURE#444,DEPARTURE_TIME#445,DEPARTURE_DELAY#446,TAXI_OUT#447,WHEELS_OFF#448,SCHEDULED_TIME#449,ELAPSED_TIME#450,AIR_TIME#451,DISTANCE#452,WHEELS_ON#453,TAXI_IN#454,SCHEDULED_ARRIVAL#455,ARRIVAL_TIME#456,ARRIVAL_DELAY#457,DIVERTED#458,... 7 more fields]

**Ejercicios**

Adjunto como recurso a esta lección están los archivos movies.csv y movie_ratings.csv. En ambos archivos las columnas están delimitadas por un pip(“|”).



Cada línea del archivo movies.csv representa a un actor que actuó en una película. Si una película tiene diez actores, habrá diez filas para esa película en particular.

In [75]:
movies = spark.read.option('sep', '|').option('inferSchema', 'true').option('header', 'true').csv("./ejercicios/movies.csv")
ratings = spark.read.option('sep', '|').option('inferSchema', 'true').option('header', 'true').csv("./ejercicios/movie_ratings.csv")

In [76]:
movies.show(truncate=False)

+-----------------+---------------------------+----+
|actor            |pelicula                   |año |
+-----------------+---------------------------+----+
|McClure, Marc (I)|Freaky Friday              |2003|
|McClure, Marc (I)|Coach Carter               |2005|
|McClure, Marc (I)|Superman II                |1980|
|McClure, Marc (I)|Apollo 13                  |1995|
|McClure, Marc (I)|Superman                   |1978|
|McClure, Marc (I)|Back to the Future         |1985|
|McClure, Marc (I)|Back to the Future Part III|1990|
|Cooper, Chris (I)|Me, Myself & Irene         |2000|
|Cooper, Chris (I)|October Sky                |1999|
|Cooper, Chris (I)|Capote                     |2005|
|Cooper, Chris (I)|The Bourne Supremacy       |2004|
|Cooper, Chris (I)|The Patriot                |2000|
|Cooper, Chris (I)|The Town                   |2010|
|Cooper, Chris (I)|Seabiscuit                 |2003|
|Cooper, Chris (I)|A Time to Kill             |1996|
|Cooper, Chris (I)|Where the Wild Things Are  

In [77]:
ratings.show(truncate=False)

+----------+--------------------------+----+
|valoracion|pelicula                  |año |
+----------+--------------------------+----+
|1.6339    |'Crocodile' Dundee II     |1988|
|7.6177    |10                        |1979|
|1.2864    |10 Things I Hate About You|1999|
|0.3243    |10,000 BC                 |2008|
|0.3376    |101 Dalmatians            |1996|
|0.5218    |102 Dalmatians            |2000|
|12.8205   |1066                      |2012|
|0.6829    |12                        |2007|
|7.4061    |12 Rounds                 |2009|
|2.3677    |127 Hours                 |2010|
|1.3585    |13 Going on 30            |2004|
|8.4034    |13 game sayawng           |2006|
|0.59      |1408                      |2007|
|4.4292    |15 Minutes                |2001|
|2.2118    |16 Blocks                 |2006|
|1.0491    |17 Again                  |2009|
|3.9265    |1941                      |1979|
|10.4757   |2 Days in the Valley      |1996|
|0.4       |2 Fast 2 Furious          |2003|
|11.1111  

1. Calcule la cantidad de películas en las que participó cada actor. La salida debe tener dos columnas: actor y conteo. La salida debe ordenarse por el conteo en orden descendente.

In [78]:
from pyspark.sql.functions import col, desc

In [79]:
movies.groupBy('actor').count().orderBy(desc('count')).show()

+-------------------+-----+
|              actor|count|
+-------------------+-----+
|   Tatasciore, Fred|   38|
|      Welker, Frank|   38|
| Jackson, Samuel L.|   32|
|      Harnell, Jess|   31|
|        Damon, Matt|   27|
|      Willis, Bruce|   27|
|  Cummings, Jim (I)|   26|
|         Hanks, Tom|   25|
|   Lynn, Sherry (I)|   25|
|    Bergen, Bob (I)|   25|
|    McGowan, Mickie|   25|
|      Proctor, Phil|   24|
|        Cruise, Tom|   23|
|         Pitt, Brad|   23|
|   Wilson, Owen (I)|   23|
|       Depp, Johnny|   22|
|Freeman, Morgan (I)|   22|
|     Morrison, Rana|   22|
|Williams, Robin (I)|   22|
|      Diaz, Cameron|   21|
+-------------------+-----+
only showing top 20 rows



2. Calcule la cantidad de películas producidas cada año. La salida debe tener tres columnas: año, siglo al que pertenece el año y conteo. La salida debe ordenarse por el conteo en orden descendente.

In [83]:
from pyspark.sql.functions import count_distinct, when, lit, countDistinct

In [90]:
movies.groupBy('año').agg(
    countDistinct('pelicula').alias('conteo')   
).withColumn('siglo', when((col('año') >= 1900) & (col('año') < 2000), lit('XX')).otherwise(lit('XXI'))).orderBy(desc('conteo')).select(
        col('año'),
        col('siglo'),
        col('conteo')
).show()

+----+-----+------+
| año|siglo|conteo|
+----+-----+------+
|2011|  XXI|    86|
|2004|  XXI|    86|
|2006|  XXI|    86|
|2005|  XXI|    85|
|2008|  XXI|    82|
|2002|  XXI|    81|
|2010|  XXI|    78|
|2000|  XXI|    77|
|2003|  XXI|    76|
|2007|  XXI|    75|
|2001|  XXI|    71|
|2009|  XXI|    68|
|1999|   XX|    67|
|1997|   XX|    66|
|1998|   XX|    59|
|1996|   XX|    42|
|2012|  XXI|    32|
|1995|   XX|    25|
|1994|   XX|    16|
|1986|   XX|    16|
+----+-----+------+
only showing top 20 rows



3. Obtenga la película con la calificación más alta por año. La salida debe tener solo una película por año y debe contener tres columnas: año, título de la película y valoración.

In [91]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, max

In [92]:
windowsSpec = Window.partitionBy('año').orderBy(desc('valoracion'))
windowsSpecAgg = Window.partitionBy('año')

In [93]:
ratings.withColumn('row_number', row_number().over(windowsSpec)).withColumn('max_val', max('valoracion').over(windowsSpecAgg)).filter(
    col('row_number') == 1
).select(
    col('año'), 
    col('pelicula'), 
    col('max_val')
).show()

+----+--------------------+-------+
| año|            pelicula|max_val|
+----+--------------------+-------+
|1937|Snow White and th...| 2.2207|
|1939|    The Wizard of Oz| 7.9215|
|1940|           Pinocchio| 7.8557|
|1942|               Bambi| 1.5053|
|1946|   Song of the South|  7.602|
|1950|          Cinderella| 9.4226|
|1953|           Peter Pan| 5.4756|
|1954|         Rear Window|10.7625|
|1955|  Lady and the Tramp| 5.1258|
|1956|Around the World ...|14.0607|
|1959|     Sleeping Beauty| 6.3919|
|1960|              Psycho|10.6375|
|1961|One Hundred and O...| 0.6726|
|1962|     The Longest Day|12.8866|
|1963|It's a Mad Mad Ma...|  6.626|
|1964|        My Fair Lady|  7.587|
|1965|      Doctor Zhivago| 4.9304|
|1966|Who's Afraid of V...|11.1111|
|1967|     The Dirty Dozen| 13.388|
|1968|        The Love Bug|13.4383|
+----+--------------------+-------+
only showing top 20 rows

